# CS 235 - Pet Adoption Speed - Random Forest Model

# Preprocessing
This preprocessing was the same done in the group, so I used Ashley Pang's preprocessing code since the decision trees are the most similar to my model.

In [1]:
import pandas as pd

# read everything in
df_train = pd.read_csv('petfinder-adoption-prediction/train/train.csv')
df_breedLabels = pd.read_csv("petfinder-adoption-prediction/PetFinder-BreedLabels.csv")
df_colorLabels = pd.read_csv("petfinder-adoption-prediction/PetFinder-ColorLabels.csv")
df_stateLabels = pd.read_csv("petfinder-adoption-prediction/PetFinder-StateLabels.csv")
# df_test = pd.read_csv('petfinder-adoption-prediction/test/test.csv')

# import pandas as pd
# from google.colab import drive

# # read everything in
# drive.mount('/content/drive/', force_remount=True)
# df_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/petfinder-adoption-prediction/train/train.csv')
# df_breedLabels = pd.read_csv('/content/drive/My Drive/Colab Notebooks/petfinder-adoption-prediction/PetFinder-BreedLabels.csv')
# df_colorLabels = pd.read_csv('/content/drive/My Drive/Colab Notebooks/petfinder-adoption-prediction/PetFinder-ColorLabels.csv')
# df_stateLabels = pd.read_csv('/content/drive/My Drive/Colab Notebooks/petfinder-adoption-prediction/PetFinder-StateLabels.csv')
# # df_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/petfinder-adoption-prediction/test/test.csv')

print(df_train)

       Type            Name  Age  Breed1  Breed2  Gender  Color1  Color2  \
0         2          Nibble    3     299       0       1       1       7   
1         2     No Name Yet    1     265       0       1       1       2   
2         1          Brisco    1     307       0       1       2       7   
3         1            Miko    4     307       0       2       1       2   
4         1          Hunter    1     307       0       1       1       0   
...     ...             ...  ...     ...     ...     ...     ...     ...   
14988     2             NaN    2     266       0       3       1       0   
14989     2  Serato & Eddie   60     265     264       3       1       4   
14990     2         Monkies    2     265     266       3       5       6   
14991     2         Ms Daym    9     266       0       2       4       7   
14992     1            Fili    1     307     307       1       2       0   

       Color3  MaturitySize  ...  Health  Quantity  Fee  State  \
0           0        

In [2]:
# preprocessing
import numpy as np
from scipy import stats

df_train = df_train.drop(['Name', 'RescuerID', 'PetID', 'Description'], axis=1)
df_breedLables = df_breedLabels.drop(['Type'], axis=1)

# Drop rows with missing information
df_train = df_train.dropna()

# Calculate z-score for only numerical columns
z_scores_fee = np.abs(stats.zscore(df_train['Fee']))
z_scores_age = np.abs(stats.zscore(df_train['Age']))

# Using a threshold of 5 std dev. identify the outlier rows
outlier_rows_fee = z_scores_fee > 5
outlier_rows_age = z_scores_age > 5
combined_outlier_rows = outlier_rows_fee | outlier_rows_age

# Remove rows with outliers
df_train = df_train[~combined_outlier_rows]
print(f"Removed {combined_outlier_rows.sum()} rows due to outliers. There are now {df_train.shape[0]} rows in our training data.")
# print(df_train.head())

# Separate the resulting column from the training data
adoption_speed_train = df_train.pop('AdoptionSpeed')

Removed 197 rows due to outliers. There are now 14796 rows in our training data.


In [3]:
# Separating categorical features
binary_categorical_columns = ['Gender','Vaccinated','Dewormed', 'Sterilized']
ordinal_categorical_columns = ['MaturitySize', 'FurLength', 'Health', 'Color1', 'Color2', 'Color3']
nonordinal_categorical_columns = ['Breed1', 'Breed2']

from sklearn.preprocessing import LabelEncoder
# Encoding
label_encoder = LabelEncoder()
df_train = pd.get_dummies(df_train, columns=binary_categorical_columns)
for column in ordinal_categorical_columns:
  df_train[column] = label_encoder.fit_transform(df_train[column])

# Split non-ordinal categorical columns into n features (similar to Benjamin's preprocessing)
n = 10

for feature in nonordinal_categorical_columns:
  top_N_values = df_train[feature].value_counts().head(n)
  print(f'Top {n} values for {feature}:\n{top_N_values}\n')

  top_N_value_names = top_N_values.index
  for index, row in df_train.iterrows():
    # If value isn't top N frequency, replace with -1 (other)
    if row[feature] not in top_N_value_names:
      df_train.at[index, feature] = -1

  df_train = pd.get_dummies(df_train, columns=[feature])

Top 10 values for Breed1:
307    5903
266    3623
265    1257
299     342
264     291
292     262
285     202
141     199
205     175
218     161
Name: Breed1, dtype: int64

Top 10 values for Breed2:
0      10613
307     1723
266      597
265      321
299      137
264      116
292      104
218       91
141       85
285       76
Name: Breed2, dtype: int64



In [4]:
from sklearn.model_selection import train_test_split
# Split 85% training, 10% test, 5% validation
df_train_85, X_temp, adoption_speed_train_85, y_temp = train_test_split(df_train, adoption_speed_train, test_size=0.15, random_state=42, stratify=adoption_speed_train)    # 85% training set, 15% temp set
df_test_10, df_val_5, adoption_speed_test_10, adoption_speed_val_5 = train_test_split(X_temp, y_temp, test_size=1/3, random_state=42, stratify=y_temp)       # 10% test set, 5% validation set

End of pre-processing.

# Random Forest Model (SKLearn)

Used for understanding Random Forest Classification:  
https://www.analyticsvidhya.com/blog/2021/06/understanding-random-forest/  
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html 

How the Random Forest Model works:
* Create n decision trees based on a subset of the training sample (using parameters).  
* Pass test data into each decision tree and choose the average/majority result from those.  

In [5]:
X = df_train_85
Y = adoption_speed_train_85

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
# from sklearn.datasets import make_classification

# Default Classifier (with random_state=2 for demonstration)
classifier = RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=2)
classifier.fit(X, Y)
print('OOB score:', classifier.oob_score_)

OOB score: 0.3928912213740458


## Default Parameters Validation

In [7]:
# method taken from Benjamin Denzler

from sklearn.metrics import precision_score, recall_score
classifier = RandomForestClassifier(n_jobs=-1, oob_score=True)		# removed random_state
classifier.fit(df_train_85, adoption_speed_train_85)
test_predictions = classifier.predict(df_test_10)

accuracy = classifier.score(df_train_85, adoption_speed_train_85)
print(f'Accuracy on training: {round(accuracy, 3) * 100}%')

accuracy = classifier.score(df_test_10, adoption_speed_test_10)
print(f'Accuracy on test: {round(accuracy, 3) * 100}%')

accuracy = classifier.score(df_val_5, adoption_speed_val_5)
print(f'Accuracy on validation: {round(accuracy, 3) * 100}%')

# zero_divison=0 returns 0 precision/recall if no positive samples for a class
avg_precision = precision_score(
    adoption_speed_test_10, test_predictions, average='weighted', zero_division=0
)
avg_recall = recall_score(
    adoption_speed_test_10, test_predictions, average='weighted', zero_division=0
)
precision_per_class = precision_score(
    adoption_speed_test_10, test_predictions, average=None, zero_division=0
)
recall_per_class = recall_score(
    adoption_speed_test_10, test_predictions, average=None, zero_division=0
)

print(f'Average weighted precision: {round(avg_precision, 3) * 100}%')
print(f'Average weighted recall: {round(avg_recall, 3) * 100}%')
for class_label, precision, recall in zip(range(len(precision_per_class)), precision_per_class, recall_per_class):
    print(f'Class {class_label}: Precision = {round(precision, 3) * 100}%, Recall = {round(recall, 3) * 100}%')

Accuracy on training: 98.4%
Accuracy on test: 39.900000000000006%
Accuracy on validation: 41.199999999999996%
Average weighted precision: 38.7%
Average weighted recall: 39.800000000000004%
Class 0: Precision = 20.0%, Recall = 5.0%
Class 1: Precision = 33.300000000000004%, Recall = 32.800000000000004%
Class 2: Precision = 37.0%, Recall = 40.1%
Class 3: Precision = 36.5%, Recall = 26.0%
Class 4: Precision = 47.9%, Recall = 58.8%


In [8]:
# validation method taken from Benjamin Denzler
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score

num_folds = 10

scoring_metrics = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(
        precision_score, average='weighted', zero_division=0
    ),
    'recall': make_scorer(
        recall_score, average='weighted', zero_division=0
    )
}

scores = cross_validate(
    classifier, df_train_85, adoption_speed_train_85,
    cv=num_folds, scoring=scoring_metrics
)

print(f'\n{num_folds}-fold validation accuracy mean: {round(scores["test_accuracy"].mean(), 3) * 100}%')
print(f'{num_folds}-fold validation precision mean: {round(scores["test_precision"].mean(), 3) * 100}%')
print(f'{num_folds}-fold validation recall mean: {round(scores["test_recall"].mean(), 3) * 100}%')


10-fold validation accuracy mean: 39.800000000000004%
10-fold validation precision mean: 38.800000000000004%
10-fold validation recall mean: 39.800000000000004%


## Hyperparameters:  
There are a few parameters that we can adjust to try to get closer to our goal.  
* n_estimators - the number of decision trees created
* max_features - the maximum number of features 
* min_samples_leaf - the minumum number of leaves required to split an internal node
* criterion - the method used to split nodes (Entropy, Gini impurity, Log Loss)
* max_leaf_nodes - maximum leaf nodes in a tree
* max_depth - limit the amount of decisions we are doing per tree

In [9]:
# Add hyperparameters to get a better accuracy
classifier = RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=2, n_estimators=200, max_features=8)
classifier.fit(X, Y)
print('OOB score:', classifier.oob_score_)

OOB score: 0.3955947837150127


Using GridSearch, all combinations of predetermined hyperparameters are tested and the highest scoring classifier is used and returned.  
n_estimators, max_features, and min_sample_leaf are iterated through. These parameters help scale the amount of information that is used when creating the decision trees.  
Through testing, entropy was found to have the most success. max_leaf_nodes and max_depth both help limit the tree so that it does not overfit. However with the parameters and dataset given, making these values infinite (=None) increased the accuracy score.  

In [10]:
classifier = RandomForestClassifier(n_jobs=-1)

n_features = len(df_train_85.columns)
#Hyperparameters
params = {											# defaults
	# 'n_estimators': [2]
	'n_estimators': [100,150,200,250],			# 100
	'min_samples_split': [1,2,5,10],			# 1
	'min_samples_leaf': [1,2,5],				# 1
	'max_depth': [3,10,20,25]						# None
}

from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(estimator=classifier, param_grid=params, cv=5, n_jobs=-1, verbose=1, scoring="accuracy")
gs.fit(X, Y)

print(gs.best_score_, gs.best_estimator_)

Fitting 5 folds for each of 192 candidates, totalling 960 fits


c:\Users\eyeba\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
240 fits failed out of a total of 960.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
240 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\eyeba\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\eyeba\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "c:\Users\eyeba\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base

0.41332769677641623 RandomForestClassifier(max_depth=25, min_samples_split=10, n_estimators=200,
                       n_jobs=-1)


# Accuracy and Precision Metrics  
These methods are used to see the accuracy, precision, and recall of our classifier. We also used a 10-fold cross validation as our validation statistic.  
The method used were programmed by Benjamin Denzler.  

In [11]:
# accuracy and precision method taken from Benjamin Denzler

from sklearn.metrics import precision_score, recall_score
best_rfm = gs.best_estimator_
best_rfm.fit(df_train_85, adoption_speed_train_85)
test_predictions = best_rfm.predict(df_test_10)

accuracy = best_rfm.score(df_train_85, adoption_speed_train_85)
print(f'Accuracy on training: {round(accuracy, 3) * 100}%')

accuracy = best_rfm.score(df_test_10, adoption_speed_test_10)
print(f'Accuracy on test: {round(accuracy, 3) * 100}%')

accuracy = best_rfm.score(df_val_5, adoption_speed_val_5)
print(f'Accuracy on validation: {round(accuracy, 3) * 100}%')

# zero_divison=0 returns 0 precision/recall if no positive samples for a class
avg_precision = precision_score(
    adoption_speed_test_10, test_predictions, average='weighted', zero_division=0
)
avg_recall = recall_score(
    adoption_speed_test_10, test_predictions, average='weighted', zero_division=0
)
precision_per_class = precision_score(
    adoption_speed_test_10, test_predictions, average=None, zero_division=0
)
recall_per_class = recall_score(
    adoption_speed_test_10, test_predictions, average=None, zero_division=0
)

print(f'Average weighted precision: {round(avg_precision, 3) * 100}%')
print(f'Average weighted recall: {round(avg_recall, 3) * 100}%')
for class_label, precision, recall in zip(range(len(precision_per_class)), precision_per_class, recall_per_class):
    print(f'Class {class_label}: Precision = {round(precision, 3) * 100}%, Recall = {round(recall, 3) * 100}%')

Accuracy on training: 82.3%
Accuracy on test: 40.0%
Accuracy on validation: 41.6%
Average weighted precision: 39.1%
Average weighted recall: 40.0%
Class 0: Precision = 50.0%, Recall = 2.5%
Class 1: Precision = 33.1%, Recall = 31.5%
Class 2: Precision = 37.1%, Recall = 39.800000000000004%
Class 3: Precision = 36.199999999999996%, Recall = 19.8%
Class 4: Precision = 46.7%, Recall = 65.9%


In [12]:
# validation method taken from Benjamin Denzler
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score

num_folds = 10

scoring_metrics = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(
        precision_score, average='weighted', zero_division=0
    ),
    'recall': make_scorer(
        recall_score, average='weighted', zero_division=0
    )
}

scores = cross_validate(
    best_rfm, df_train_85, adoption_speed_train_85,
    cv=num_folds, scoring=scoring_metrics
)

print(f'\n{num_folds}-fold validation accuracy mean: {round(scores["test_accuracy"].mean(), 3) * 100}%')
print(f'{num_folds}-fold validation precision mean: {round(scores["test_precision"].mean(), 3) * 100}%')
print(f'{num_folds}-fold validation recall mean: {round(scores["test_recall"].mean(), 3) * 100}%')


10-fold validation accuracy mean: 40.9%
10-fold validation precision mean: 40.300000000000004%
10-fold validation recall mean: 40.9%


# Implementation

## Decision Tree Implementation from Ashley Pang  
https://colab.research.google.com/drive/1yHBau067-yP7q6b1TWQAmEk7WgKNaRdW#scrollTo=kxgZYzx3Y7j-&line=62&uniqifier=1

Citations:
- [SciKitLearn - Libraries for Decision Trees](https://scikit-learn.org/stable/modules/tree.html)
- [SciKitLearn - Decision Tree Structure](https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html)
- [Datacamp -  Decision Tree Classification](https://www.datacamp.com/tutorial/decision-tree-classification-python) (For determining that CART uses the Gini index)
- [GeeksforGeeks - Decision Tree](https://www.geeksforgeeks.org/decision-tree-implementation-python/)
- [GeeksforGeeks - Gini Index](https://www.geeksforgeeks.org/gini-impurity-and-entropy-in-decision-tree-ml/)
- [Youtube video on Decision Tree Classification from scratch](https://www.youtube.com/watch?v=sgQAhG5Q7iY&t=892s)
- [GeeksforGeeks - Binary Tree Data Structure](https://www.geeksforgeeks.org/binary-tree-data-structure/)
- [Auantinsti - How to Calculate Gini Index](https://blog.quantinsti.com/gini-index/)
- [Baeldung - Splitting by Gini Index](https://www.baeldung.com/cs/impurity-entropy-gini-index)

---

In [13]:
# Setup Libraries
import numpy as np

# Implement a Decision Tree from Scratch
  # Hyperparameters to implement: max_depth=10 min_samples_split=2 min_samples_leaf=2
  # Fit using these datasets: df_train_85, adoption_speed_train_85
  # Predict the train, val and test sets (df_train_85, df_val_5, df_test_10)

# Given the features (x) and correspondings class labels (y) (0-4) return the feature with the lowest gini index
def FindMinGiniIndex(x, y):
  # Get the num of columns in x
  numOfColumns = 1
  if x.ndim > 1:
    numOfColumns = x.shape[1]

  # Initialize all gini indexes to 1 which is the worst value
  giniValues = np.ones(numOfColumns)

  for curFeature in range(numOfColumns):
    # Save all of the unique values in the column (look at all rows) and return the counts
    curColumn = x[:]
    if x.ndim > 1:
      curColumn = x[:,curFeature]

    # Remove any "Not a number" values for this column in x and in the result y
    validRows = ~np.isnan(curColumn)
    curColumnFiltered = curColumn[validRows]
    yFiltered = y[validRows]

    uniqueValues, counts = np.unique(curColumnFiltered, return_counts=True)

    if len(uniqueValues) > 0:
      curColGiniValues = np.ones(len(uniqueValues))
      # Find the probability of the uniqueValue also being in a certain class
      for i, uniqueVal in enumerate(uniqueValues):
        # Use a dictionary to count occurences of each class
        classCounts = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}
        for j, curVal in enumerate(curColumnFiltered):
          if uniqueVal == curVal and y[j] in classCounts:
            classCounts[yFiltered[j]] += 1

        # Calculate the sum of squared probabilies of each class
        probabilitySum = 0.0
        totalCount = counts[i]
        if totalCount > 0:
          for j, curClassCount in classCounts.items():
            probabilitySum += pow((curClassCount / totalCount), 2)
        else:
          probabilitySum = 0.0

        # Calculate the current gini index for this unique value and save it in an array
        # Gini Index = 1 - sum (squared probabilities of each outcome)
        curColGiniValues[i] = 1 - probabilitySum

      # Find the weighted sum of the gini indices for each unique Value
      giniValues[curFeature] = 0
      for i, curGiniVal in enumerate(curColGiniValues):
        probOfCurUniqueValue = counts[i] / len(curColumnFiltered)
        giniValues[curFeature] += (probOfCurUniqueValue) * curGiniVal

  # Find the minimum gini index
  minimumGiniIndex = np.min(giniValues)
  featureToSplit = np.argmin(giniValues)

  return minimumGiniIndex, featureToSplit

def GetBestSplitUsingGiniIndex(x, y, min_samples_leaf):
  # Find the feature to split
  minimumGiniIndex, featureToSplit = FindMinGiniIndex(x, y)
  colToSplit = x[:]
  if x.ndim > 1:
    colToSplit = x[:,featureToSplit]

  uniqueValues = np.unique(colToSplit)
  uniqueValues = np.sort(uniqueValues)
  xLeftFinal = np.array([])
  yLeftFinal = np.array([])
  xRightFinal = np.array([])
  yRightFinal = np.array([])
  minimumGini = 1
  bestThresh = uniqueValues[0]

  # Find best threshold to split by
  for i in range(1, len(uniqueValues)):
    threshold = (uniqueValues[i - 1] / 2.0) + (uniqueValues[i] / 2.0)

    # Split using the suggested threshold (average between two values in the array)
    xLeft = colToSplit[colToSplit <= threshold]
    yLeft = y[colToSplit <= threshold]
    xRight = colToSplit[colToSplit > threshold]
    yRight = y[colToSplit > threshold]
    currMingini = 1

    if ((len(xLeft) >= min_samples_leaf) and (len(xRight) >= min_samples_leaf)):
      # Adjust the x to have left on the first column and right on the second column with filler None's:
      #  xLeft = [[1]], xRight = [[15],[18]] then
      #  xCombined = [[ 1     None]
      #               [ None  15  ]
      #               [ None  18  ]]
      xLeftFiller = np.empty_like(xRight, dtype=float)
      xLeftFiller.fill(np.nan)
      xRightFiller = np.empty_like(xLeft, dtype=float)
      xRightFiller.fill(np.nan)
      xLeftTemp = np.concatenate((xLeft, xLeftFiller))
      xRightTemp = np.concatenate((xRightFiller, xRight))
      xCombined = np.zeros((len(xRightFiller) + len(xRight), 2))

      xCombined[:, 0] = xLeftTemp
      xCombined[:, 1] = xRightTemp

      # Adjust y to just be on top of the other:
      #  yLeft = [[1],[2]], yRight = [[3]] then
      #  yCombined = [[1]
      #               [2]
      #               [3]]
      yCombined = np.concatenate((yLeft, yRight))

      # Calculate the gini index given first column is xLeft and second column is xRight
      currMingini, column = FindMinGiniIndex(xCombined, yCombined)

    # Keep updating the minimumGini and keep the best splits
    if minimumGini > currMingini:
      minimumGini = currMingini
      bestThresh = threshold

  validLeftRows = x[:] <= bestThresh
  validRightRows = x[:] > bestThresh
  if x.ndim > 1:
    validLeftRows = x[:, featureToSplit] <= bestThresh
    validRightRows = x[:, featureToSplit] > bestThresh

  xLeftFinal = x[validLeftRows]
  yLeftFinal = y[validLeftRows]
  xRightFinal = x[validRightRows]
  yRightFinal = y[validRightRows]

  return xLeftFinal, yLeftFinal, xRightFinal, yRightFinal, featureToSplit, minimumGini, bestThresh

def printSplit(xL, yL, xR, yR, split, gini, thresh):
  print(f"Split by feature column {split}, with gini index = {gini} and threshold <= {thresh}")

  print("XLeft:")
  print(xL)
  print("YLeft:")
  print(yL)
  print("\nXRight:")
  print(xR)
  print("YRight:")
  print(yR)
  print("\n")

class DecisionTreeNode:
  def __init__(self, num, features, labels, featureToSplit, minimumGini, bestThresh):
    self.x = features
    self.y = labels
    self.childLeft = None
    self.childRight = None
    self.feature = featureToSplit
    self.giniIndex = minimumGini
    self.threshold = bestThresh
    self.majorityClass = self.calculateMajorityClass()

  # Print the node and its children similar to sklearn's exportText function
  def printNode(self, indent=""):
    if not self.isLeaf():
      print(f"{indent}|--- {self.feature} <= {self.threshold}, giniIndex = {self.giniIndex}")
      if self.childLeft is not None and self.childRight is not None:
        if self.childLeft.isLeaf():
          print(f"{indent}|  |--- Class: {self.childLeft.majorityClass}")
        else:
          self.childLeft.printNode(indent + "|  ")
      print(f"{indent}|--- {self.feature} > {self.threshold}, giniIndex = {self.giniIndex}")
      if self.childLeft is not None and self.childRight is not None:
        if self.childRight.isLeaf():
          print(f"{indent}|  |--- Class: {self.childRight.majorityClass}")
        else:
          self.childRight.printNode(indent + "|  ")

  # For predicting, we assign each node with a class based on majority
  def calculateMajorityClass(self):
    if len(np.bincount(self.y)) > 0:            # needed as a change for validation
      return np.argmax(np.bincount(self.y))
    else:
      return -1

  # Boolean check to see if the current leaf is a child
  def isLeaf(self):
    return self.childLeft is None and self.childRight is None or (self.feature is None and self.threshold is None)

# Build the tree given the hyperparameters
class DecisionTree:
  # Initialize the tree with nothing in all parameters
  def __init__(self, root=None, numNodes=0, max_depth=None, min_samples_split=None, min_samples_leaf=None, columnNames=None):
    self.root = root
    self.numNodes = numNodes
    self.columnNames = columnNames
    self.max_depth = max_depth
    self.min_samples_split = min_samples_split
    self.min_samples_leaf = min_samples_leaf

  # Define a fit function that builds the tree and uses the recursive call (starts with the root)
  def fit(self, x, y):
    if not isinstance(x, np.ndarray):
      x = x.to_numpy()
    if not isinstance(y, np.ndarray):
      y = y.to_numpy()
    # Call this once and it will create the full tree
    self.root = self.BuildTreeRecursive(x, y, self.max_depth, self.min_samples_split, self.min_samples_leaf, self.columnNames, 0)

  # The recursive function that will stop once we meet the hyperparameter conditions
  def BuildTreeRecursive(self, x, y, max_depth, min_samples_split, min_samples_leaf, columnNames, depth):
    # Stop if we reached max_depth, or not enough samples to split or if everything in y is one class
    if depth == max_depth or len(x) < min_samples_split or len(np.unique(y)) == 1:
      # This is a child node
      return DecisionTreeNode(self.numNodes, x, y, None, None, None)

    else:
      # Define the current node which is the parent
      XLeft, YLeft, XRight, YRight, featureToSplit, minimumGini, bestThresh  = GetBestSplitUsingGiniIndex(x, y, min_samples_leaf)
      parentNode = DecisionTreeNode(self.numNodes, x, y, columnNames[featureToSplit], minimumGini, bestThresh)
      self.numNodes += 1

      # Recursively create the children (left and right)
      parentNode.childLeft = self.BuildTreeRecursive(XLeft, YLeft, max_depth, min_samples_split, min_samples_leaf, columnNames, depth + 1)
      parentNode.childRight = self.BuildTreeRecursive(XRight, YRight, max_depth, min_samples_split, min_samples_leaf, columnNames, depth + 1)

      # After returning from all of the recursive calls, return the root parent node
      return parentNode

  # Return an array of predictions of what class the row should be
  def predict(self, features):
    if not isinstance(features, np.ndarray):
      features = features.to_numpy()
    numRows = features.shape[0]
    prediction = np.zeros((numRows, 1))

    # Save prediction after tracing through the tree
    for row in range(numRows):
      curNode = self.root

      # Check every non-leaf node and keep going through the branches
      while not curNode.isLeaf():
        # Make sure that the features passed in are what the tree was trained on
        try:
          column = self.columnNames.index(curNode.feature)
        except ValueError:
          raise ValueError(f"Feature '{curNode.feature}' not found in trained DecisionTree.")
        if features[row, column] <= curNode.threshold:
          curNode = curNode.childLeft
        else:
          curNode = curNode.childRight

      # This node's prediction is based on its majorityClass
      prediction[row, 0] = curNode.majorityClass

    return prediction

  # Calls print on the root and that node's print function will recursively call the rest of the node's prints
  def PrintTree(self):
    if self.root is not None:
      self.root.printNode()

## Random Forest Implementation

The main methodology for the random forest is splitting the data that is passed into the decision trees. The decision tree classifier was implemented by Ashley Pang, so all that needed to be done was the bagging method.  


Reference page: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html  
Setting params to dict: https://stackoverflow.com/questions/64918714/fastest-way-to-assign-variables-to-python-class  
Random Forest from Scratch Reference: https://machinelearningmastery.com/implement-random-forest-scratch-python/  

In [14]:
from math import sqrt
from random import seed
from random import randrange
import multiprocessing
# use:
# classifier = RandForest(n_jobs=-1, oob_score=True, random_state=2)
# classifier.fit(X, Y)
class RandForest:
	hyperparams = {}
	forest = list()
	oob_score = 0.0

	def __init__(self, n_estimators=100, max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features=None, oob_score=False, random_state=None):
		self.hyperparams['n_estimators']=n_estimators				# number of trees created
		self.hyperparams['max_depth']=max_depth					# implemented from decision tree
		self.hyperparams['min_samples_split']=min_samples_split	# implemented from decision tree
		self.hyperparams['min_samples_leaf']=min_samples_leaf		# implemented from decision tree
		self.hyperparams['max_features']=max_features
		self.hyperparams['oob_score']=oob_score					# accuracy score
		self.hyperparams['random_state']=random_state				# random state (seed)
	
	#used for GridSearch
	def get_params(self, deep=True):
		return self.hyperparams
	def set_params(self, **params):
		for i in params.keys():
			self.hyperparams[i] = params[i]
		return self
	# Calculate accuracy percentage
	def accuracy(self, actual, predict):
		correct = 0
		for i in range(len(actual)):
			if actual.iloc[i] == predict[i]:
				correct += 1
		return correct / float(len(actual)) * 100.0

	# create subsample using ratio
	def subsample(self, data, data_class, ratio):
		sample = pd.DataFrame(columns=X.columns)
		sample_class = list()
		sample_size = round(len(data)*ratio)
		while len(sample) < sample_size:
			index = randrange(0, len(data.index))
			sample.loc[len(sample.index)] = data.iloc[index]	#https://www.geeksforgeeks.org/how-to-add-one-row-in-an-existing-pandas-dataframe/
			sample_class.append(data_class.iloc[index])
		return sample, pd.Series(sample_class)
	
	# for all trees, get result from each, output max for each row
	# https://www.educative.io/answers/find-the-maximum-value-in-each-row-and-column-of-a-numpy-2d-array
	def bagging(self, features):
		predictions = list()
		for tree in self.forest:
			p = tree.predict(features)
			predictions.append(p)
		results = np.amax(np.array(predictions), axis = 0)
		return results
	
	# fit forest trees with subsamples
	def fit(self, X, Y, sample_ratio=0.8):
		if self.hyperparams['random_state'] != None:
			seed(self.hyperparams['random_state'])
		self.forest = list()
		for i in range(self.hyperparams['n_estimators']):
			sample, sample_class = self.subsample(X, Y, sample_ratio)
			tree = DecisionTree(max_depth=self.hyperparams['max_depth'], 
					   			min_samples_split=self.hyperparams['min_samples_split'], 
								min_samples_leaf=self.hyperparams['min_samples_leaf'], 
								columnNames=X.columns.tolist())
			tree.fit(sample, sample_class)
			self.forest.append(tree)
		if self.hyperparams['oob_score'] == True:
			predictions = self.predict(X).tolist()
			self.oob_score = self.accuracy(Y, predictions)
		return self

	# given inputs (rows with features), output array of predicted values
	def predict(self, features):
		if not isinstance(features, np.ndarray):
			features = features.to_numpy()
		return self.bagging(features)

* RandForest() is initialize with hyper-parameters, although these can be changed/obtained using set_params() and get_params() respectively.
* We have a dictionary to store the hyper-parameters as well as the forest of decision trees created during fitting and the oob_score.
* We have subsample(), which will return a subsample of the data passed in, as well as the results.
* bagging() will aggregate all of the forest's prediction results and return the array of predictions for each row of the input.
* fit() will use the X and Y inputs to create n_estimator decision trees based on the hyper-parameters. If oob_score is set to true, we can also spend time calculating the accuracy of this fit based on itself.
* accuracy() will iterate through all actual and predicted results and return the fraction of results that turned out to be correct.
* predict() will take the input feature set and pass it into the bagging function. THe result is output.

In [15]:
# Implementation using Ashley Pang's chosen hyperparameters (using two trees)
classifier = RandForest(n_estimators=2,max_depth=10,min_samples_split=2,min_samples_leaf=2,oob_score=True, random_state=2)
classifier.fit(X,Y)
classifier.oob_score

34.43066157760814

In [16]:
# Use a sample of n_estimator values to find the best one to use
# classifier = RandForest(max_depth=10,min_samples_split=2,min_samples_leaf=2,oob_score=True)
n_features = len(df_train_85.columns)

est_val = [2,5,10]
scores = list()
for i in est_val:
	classifier = RandForest(n_estimators=i,max_depth=10,min_samples_split=2,min_samples_leaf=2,oob_score=True)
	classifier.fit(X,Y)
	scores.append(classifier.oob_score)
n_est = max(scores)
print(n_est,scores.index(n_est))

35.06679389312977 2


Validation method taken from Ashley Pang and Benjamin Denzler

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

def cross_validate_manual(model, x, y, cv, scoring, random_seed=None):
  # Initialization of the used variables
  accuracy_list = []
  precision_list = []
  recall_list = []
  np.random.seed(random_seed)

  # Per split
  for i in range(cv):
    # set the random seed and then split using it
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=1/cv, random_state=random_seed)
    model.fit(x_train, y_train)
    predictions = model.predict(x_val)

    # Look at each metric that we have passed in and choose a scoring function, only works for the three below
    for metric in scoring:
      if metric == 'accuracy':
        accuracyScore = accuracy_score(y_val, predictions)
        accuracy_list.append(accuracyScore)
      elif metric == 'precision':
        precisionScore = precision_score(y_val, predictions, average='weighted', zero_division=0)
        precision_list.append(precisionScore)
      elif metric == 'recall':
        recallScore = recall_score(y_val, predictions, average='weighted', zero_division=0)
        recall_list.append(recallScore)

  # Populate the result with the three lists kept
  result_dictionary = {
    'test_accuracy': accuracy_list,
    'test_precision': precision_list,
    'test_recall': recall_list
  }

  return result_dictionary


# Perform Cross Valdiation on default and pre-pruned decision trees
# Uses skLearn's model with Ashley's cross validation function
scoring_metrics = ['accuracy','precision','recall']
scores_skLearn = cross_validate_manual(RandomForestClassifier(), df_train_85, adoption_speed_train_85, cv=10, scoring=scoring_metrics)
print("Default")
print(f'10-fold validation accuracy mean: {np.mean(scores_skLearn["test_accuracy"])}')
print(f'10-fold validation precision mean: {np.mean(scores_skLearn["test_precision"])}')
print(f'10-fold validation recall mean: {np.mean(scores_skLearn["test_recall"])}')

scoring_metrics = ['accuracy','precision','recall']
# scores_skLearn = cross_validate_manual(RandomForestClassifier(n_estimators=300,min_samples_split=2,min_samples_leaf=5), df_train_85, adoption_speed_train_85, cv=10, scoring=scoring_metrics)
scores_skLearn = cross_validate_manual(gs.best_estimator_, df_train_85, adoption_speed_train_85, cv=10, scoring=scoring_metrics)
print("Pruned")
print(f'10-fold validation accuracy mean: {np.mean(scores_skLearn["test_accuracy"])}')
print(f'10-fold validation precision mean: {np.mean(scores_skLearn["test_precision"])}')
print(f'10-fold validation recall mean: {np.mean(scores_skLearn["test_recall"])}')

# Uses a manual implementation of cross validation and model
# Parameters: (model, x, y, cv, scoring)
scoring_metrics = ['accuracy','precision','recall']
scores_manualImp = cross_validate_manual(RandForest(n_estimators=10,max_depth=10,min_samples_split=2,min_samples_leaf=2), df_train_85, adoption_speed_train_85, cv=10, scoring=scoring_metrics)
print("Implementation")
print(f'\n10-fold validation accuracy mean: {np.mean(scores_manualImp["test_accuracy"])}')
print(f'10-fold validation precision mean: {np.mean(scores_manualImp["test_precision"])}')
print(f'10-fold validation recall mean: {np.mean(scores_manualImp["test_recall"])}')

Default
10-fold validation accuracy mean: 0.39475357710651826
10-fold validation precision mean: 0.3852302876434262
10-fold validation recall mean: 0.39475357710651826
Pruned
10-fold validation accuracy mean: 0.40802861685214625
10-fold validation precision mean: 0.40272442075465575
10-fold validation recall mean: 0.40802861685214625
Implementation

10-fold validation accuracy mean: 0.3491255961844197
10-fold validation precision mean: 0.29488343192544464
10-fold validation recall mean: 0.3491255961844197


In [18]:
# Taken from Benjamin Denzler
# Perform T-test for statistical significance between sklearn and manual implementation with hyperparameters
t_stat_accuracy, p_value_accuracy = stats.ttest_rel(scores_skLearn["test_accuracy"], scores_manualImp["test_accuracy"])
t_stat_precision, p_value_precision = stats.ttest_rel(scores_skLearn["test_precision"], scores_manualImp["test_precision"])
t_stat_recall, p_value_recall = stats.ttest_rel(scores_skLearn["test_recall"], scores_manualImp["test_recall"])

print(f'p_value_accuracy = {p_value_accuracy}')
print(f'p_value_precision = {p_value_precision}')
print(f'p_value_recall = {p_value_recall}')

p_value_accuracy = 1.4729835726087444e-05
p_value_precision = 5.766159235844788e-06
p_value_recall = 1.4729835726087444e-05
